In [1]:
import statsmodels.api as sm
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from category_encoders import TargetEncoder

In [2]:
#read in data
accidents = pd.read_csv('../../data/MA3_finished.csv', index_col='ID')
accidents = accidents.sample(frac=0.1)

In [3]:
#look at data
print(accidents.head)

<bound method NDFrame.head of          Unnamed: 0  Severity           Start_Time             End_Time  \
ID                                                                        
2486886     2486886         2  2018-06-14 16:09:59  2018-06-14 17:09:37   
772867       772867         2  2021-09-20 06:52:39  2021-09-20 07:22:13   
3522166     3522166         2  2022-03-10 16:10:06  2022-03-10 16:38:59   
4839643     4839643         2  2022-08-27 12:31:41  2022-08-27 13:34:28   
5365303     5365303         2  2021-07-29 08:10:35  2021-07-29 08:36:00   
...             ...       ...                  ...                  ...   
6473153     6473153         4  2020-03-20 18:02:35  2020-03-20 18:37:33   
1315603     1315603         2  2020-05-30 10:55:23  2020-05-30 12:29:35   
4169049     4169049         4  2022-11-05 21:17:00  2022-11-06 01:00:49   
3340293     3340293         2  2022-10-07 05:53:00  2022-10-07 08:02:32   
4394540     4394540         2  2022-09-23 16:58:05  2022-09-23 20:30:0

In [4]:
#create copy for this file and drop unnamed column name
regression_df = accidents.copy()
regression_df.drop('Unnamed: 0', axis=1, inplace=True)
#check new df
regression_df.head()

,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,City,County,...,Nautical_Twilight,Astronomical_Twilight,Year,Month,Day,Hour,Duration,State Name,Region,Division
ID,,,,,,,,,,,,,,,,,,,,,
2486886,2,2018-06-14 16:09:59,2018-06-14 17:09:37,30.676197,-88.127373,0.000,Accident on County Hwy-56 Airport Blvd Westbou...,I-65 S,Mobile,Mobile,...,Day,Day,2018,6,3,16,3578.0,Alabama,South,East South Central
772867,2,2021-09-20 06:52:39,2021-09-20 07:22:13,33.647110,-84.397804,0.000,Right hand shoulder blocked due to accident on...,GA-401 N,Atlanta,Clayton,...,Day,Day,2021,9,0,6,1774.0,Georgia,South,South Atlantic
3522166,2,2022-03-10 16:10:06,2022-03-10 16:38:59,35.269857,-80.794350,0.464,Stationary traffic from US-29/NC-49/N Tryon St...,Munsee St,Charlotte,Mecklenburg,...,Day,Day,2022,3,3,16,1733.0,North Carolina,South,South Atlantic
4839643,2,2022-08-27 12:31:41,2022-08-27 13:34:28,30.378275,-97.688442,0.278,Slow traffic on TX-275 Loop from W Grady Dr (N...,N Lamar Blvd,Austin,Travis,...,Day,Day,2022,8,5,12,3767.0,Texas,South,West South Central
5365303,2,2021-07-29 08:10:35,2021-07-29 08:36:00,33.705430,-117.855026,1.939,Slow traffic on Costa Mesa Fwy S - CA-55 S fro...,CA-55 S,Santa Ana,Orange,...,Day,Day,2021,7,3,8,1525.0,California,West,Pacific


In [5]:
#describe the data
regression_df.describe()

,Severity,Start_Lat,Start_Lng,Distance(mi),Temperature(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Year,Month,Day,Hour,Duration
count,705156.000000,705156.000000,705156.000000,705156.000000,705156.000000,705156.000000,705156.000000,705156.000000,705156.000000,705156.000000,705156.000000,705156.000000,705156.000000,7.051560e+05
mean,2.202698,36.193256,-94.567841,0.569845,61.902302,64.376226,29.514160,9.101926,7.691020,2020.028401,6.683036,2.599813,12.439467,2.609651e+04
std,0.480788,5.117869,17.306861,1.730488,19.042470,22.777913,1.014116,2.643746,5.563616,1.865549,3.635368,1.806146,5.428459,7.956168e+05
min,1.000000,24.554800,-124.535893,0.000000,-35.000000,1.000000,2.990000,0.000000,0.000000,2016.000000,1.000000,0.000000,0.000000,1.800000e+02
25%,2.000000,33.315374,-117.159039,0.000000,49.500000,48.000000,29.340000,10.000000,4.600000,2019.000000,3.000000,1.000000,8.000000,2.059000e+03
50%,2.000000,35.819729,-87.664725,0.037000,64.000000,66.000000,29.840000,10.000000,7.000000,2020.000000,7.000000,3.000000,13.000000,4.502000e+03
75%,2.000000,40.121761,-80.369779,0.479000,76.000000,84.000000,30.020000,10.000000,10.400000,2022.000000,10.000000,4.000000,17.000000,7.546000e+03
max,4.000000,48.999569,-68.132190,175.679993,196.000000,100.000000,58.390000,100.000000,822.800000,2023.000000,12.000000,6.000000,23.000000,1.341813e+08


In [6]:
#Wind_Speed 822 mph --> outliner
#print(regression_df.index[accidents ['Wind_Speed(mph)'] >822].tolist())
#print(regression_df.loc[[86481]])
#regression_df.drop(labels=[86481],axis=0, inplace=True)

In [7]:
#Wind_Speed 243 mph --> outliner
#print(regression_df.index[regression_df ['Wind_Speed(mph)'] >242].tolist())
#print (regression_df.loc[[776748]])
#regression_df.drop(labels=[776748],axis=0, inplace=True)

In [8]:
regression_df.dtypes

Severity                   int64
Start_Time                object
End_Time                  object
Start_Lat                float64
Start_Lng                float64
Distance(mi)             float64
Description               object
Street                    object
City                      object
County                    object
State                     object
Zipcode                   object
Country                   object
Timezone                  object
Airport_Code              object
Temperature(F)           float64
Humidity(%)              float64
Pressure(in)             float64
Visibility(mi)           float64
Wind_Direction            object
Wind_Speed(mph)          float64
Weather_Condition         object
Amenity                     bool
Bump                        bool
Crossing                    bool
Give_Way                    bool
Junction                    bool
No_Exit                     bool
Railway                     bool
Roundabout                  bool
Station   

In [9]:
#Rename Columns with () or ' ' for the equation --> otherwise it leads to PatsyError
regression_df = regression_df.rename(columns={
    'Temperature(F)': 'Temperature',
    'Humidity(%)': 'Humidity',
    'Visibility(mi)': 'Visibility',
    'Pressure(in)': 'Pressure',
    'Wind_Speed(mph)': 'Wind_Speed',
    'Distance(mi)': 'Distance',
    'State Name': 'StateName',
})

In [16]:
#basic linearRegressionModel
severity_basic = ols("Severity ~ Start_Lat + Start_Lng", regression_df).fit()
print(severity_basic.summary())

                            OLS Regression Results                            
Dep. Variable:               Severity   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     3220.
Date:                Fri, 05 Jan 2024   Prob (F-statistic):               0.00
Time:                        13:52:16   Log-Likelihood:            -4.8096e+05
No. Observations:              705156   AIC:                         9.619e+05
Df Residuals:                  705153   BIC:                         9.620e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.1086      0.005    425.711      0.0

In [15]:
severity_full = ols("Severity ~ Start_Lat + Start_Lng + Distance + Temperature + Humidity + Pressure + Visibility + Wind_Speed + Amenity + Bump + Crossing + Give_Way + Junction + No_Exit + Railway + Roundabout + Station + Stop + Traffic_Calming + Traffic_Signal + Turning_Loop + Year + Month + Day + Hour + Duration", regression_df).fit()
print(severity_full.summary())

                            OLS Regression Results                            
Dep. Variable:               Severity   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     3214.
Date:                Fri, 05 Jan 2024   Prob (F-statistic):               0.00
Time:                        13:49:38   Log-Likelihood:            -4.4611e+05
No. Observations:              705156   AIC:                         8.923e+05
Df Residuals:                  705130   BIC:                         8.926e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 